# 01. Project 
### IPL 2022 SQUAD SELECTION


In [1]:
import numpy as np
import pandas as pd


perf = pd.read_csv("cricket_data.csv")
auction = pd.read_csv("ipl2022 - soldplayersipl2022.csv")

print(perf.shape)
print(auction.shape)

(1172, 25)
(204, 5)


In [2]:
perf['Year'] = perf['Year'].astype(str).str.strip()
perf_2022 = perf[perf['Year'] == '2022'].copy()
print(perf_2022.shape)
perf_2022.head()

(141, 25)


,Year,Player_Name,Matches_Batted,Not_Outs,Runs_Scored,Highest_Score,Batting_Average,Balls_Faced,Batting_Strike_Rate,Centuries,...,Matches_Bowled,Balls_Bowled,Runs_Conceded,Wickets_Taken,Best_Bowling_Match,Bowling_Average,Economy_Rate,Bowling_Strike_Rate,Four_Wicket_Hauls,Five_Wicket_Hauls
2,2022,Ruturaj Gaikwad,14,0,368,99,26.29,291,126.46,0,...,14,0,0,0,0,0,0,0,0,0
6,2022,Devon Conway,7,1,252,87,42,173,145.66,0,...,7,0,0,0,0,0,0,0,0,0
9,2022,MS Dhoni,14,6,232,50*,33.14,188,123.40,0,...,14,0,0,0,0,0,0,0,0,0
26,2022,Ajinkya Rahane,7,0,133,44,19,128,103.91,0,...,7,0,0,0,0,0,0,0,0,0
45,2022,Ravindra Jadeja,10,4,116,26*,19.33,98,118.37,0,...,10,198,248,5,3/39,49.60,7.52,39.60,0,0


In [3]:
auction['Price_in_Cr'] = (auction['Price Paid'].str.replace('₹','', regex=False).str.replace(',','', regex=False).astype(float)/ 1e7)
auction[['Players','Type','Price_in_Cr']].head()

perf_2022['player'] = perf_2022['Player_Name'].str.lower().str.strip()
auction['player'] = auction['Players'].str.lower().str.strip()


df = pd.merge(perf_2022, auction[['player','Type','Price_in_Cr']], on='player', how='inner')
print(df.shape)
df[['player','Type','Price_in_Cr']].head()

(95, 28)


,player,Type,Price_in_Cr
0,devon conway,Batsman,1.00
1,ajinkya rahane,Batsman,1.00
2,mitchell santner,All-Rounder,1.90
3,shivam dube,All-Rounder,4.00
4,shardul thakur,Bowler,10.75


In [4]:
## normalizing function

def normalise(s):
    if s.max()==s.min():
        return 0
    else:
        return ((s-s.min())/(s.max()-s.min()))

In [5]:
numeric_cols=['Runs_Scored',
    'Batting_Strike_Rate',
    'Batting_Average',
    'Fours',
    'Sixes',
    'Wickets_Taken',
    'Economy_Rate',
    'Bowling_Average',
    'Bowling_Strike_Rate',
    'Four_Wicket_Hauls',
    'Five_Wicket_Hauls',
    'Catches_Taken',
    'Stumpings',
    'Matches_Batted'
    
]
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [6]:
###   Batting perfomance   


df['Boundaries'] = normalise(df['Fours'] + df['Sixes']) 

df = df[df['Matches_Batted'] > 0].copy()
df['Runs_per_match'] = df['Runs_Scored'] / df['Matches_Batted']

df['Runs'] = normalise(df['Runs_per_match'])
df['Bat_Avg'] = normalise(df['Batting_Average'])
df['Bat_strk'] = normalise(df['Batting_Strike_Rate'])

df['bat_score'] = (0.30*df['Runs'] + 0.30*df['Bat_strk'] + 0.25*df['Bat_Avg'] + 0.15*df['Boundaries'])


In [7]:
###   Bowling perfomance

df['Hauls'] = df['Four_Wicket_Hauls'] + df['Five_Wicket_Hauls']

df['Hauls'] = normalise(df['Hauls'])
df['Economy_Rate'] = normalise(df['Economy_Rate'])
df['Eco_inv'] = 1 - df['Economy_Rate']
df['Bowling_Average'] = normalise(df['Bowling_Average'])
df['Bowl_Avg_inv'] = 1 - df['Bowling_Average'] 
df['Bowling_Strike_Rate'] = normalise(df['Bowling_Strike_Rate'])
df['Bowl_strk_inv'] = 1 - df['Bowling_Strike_Rate']

df['bowl_score'] = (0.30*df['Bowl_Avg_inv'] + 0.30*df['Eco_inv'] + 0.20*df['Bowl_strk_inv'] + 0.20*df['Hauls'])



In [8]:
## WicketKeeper perfomance

df['Catches_Taken'] = normalise(df['Catches_Taken'])
df['Stumpings'] = normalise(df['Stumpings'])

df['Keeper_Score'] = (0.4*df['Catches_Taken'] + 0.6*df['Stumpings'])

In [9]:
## Final Score

def final_score(row):
    role = row['Type']

    if role == 'Batsman':
        return row['bat_score']
    elif role == 'Bowler':
        return row['bowl_score']
    elif role == 'Wicket Keeper':
        return (0.7*row['bat_score'] + 0.3*row['Keeper_Score'])
    else:   # All-Rounder
        return 0.6 * max(row['bat_score'], row['bowl_score']) + 0.4 * min(row['bat_score'], row['bowl_score'])

        
df['final_score'] = df.apply(final_score, axis=1)
df['value_per_crore'] = df['final_score'] / df['Price_in_Cr']


In [10]:
df[['player','Type','Price_in_Cr','final_score','value_per_crore']].sort_values(by='value_per_crore', ascending=False).head()

,player,Type,Price_in_Cr,final_score,value_per_crore
45,mohsin khan,Bowler,0.2,0.707402,3.537008
16,rasikh dar,Bowler,0.2,0.683333,3.416667
20,b. sai sudharsan,All-Rounder,0.2,0.682153,3.410765
42,prerak mankad,All-Rounder,0.2,0.614190,3.070952
7,mukesh choudhary,Bowler,0.2,0.598327,2.991634


In [11]:
# Selection List

df['price_norm'] = (df['Price_in_Cr'] - df['Price_in_Cr'].min()) /(df['Price_in_Cr'].max() - df['Price_in_Cr'].min())

df['price_inv'] = 1 - df['price_norm']
df['balanced_value'] = 0.7*df['final_score'] + 0.3*df['price_inv']

df_sorted = df.sort_values(by='balanced_value', ascending=False).reset_index(drop=True)
df_sorted[['player','Type','Price_in_Cr','balanced_value']]

,player,Type,Price_in_Cr,balanced_value
0,mohsin khan,Bowler,0.20,0.795181
1,kuldeep yadav,Bowler,2.00,0.785678
2,rasikh dar,Bowler,0.20,0.778333
3,b. sai sudharsan,All-Rounder,0.20,0.777507
4,david miller,Batsman,3.00,0.751984
...,...,...,...,...
90,david willey,All-Rounder,2.00,0.396339
91,washington sundar,All-Rounder,8.75,0.389233
92,kartik tyagi,Bowler,4.00,0.369668
93,ishan kishan,Wicket Keeper,15.25,0.346411


In [12]:
MIN_ROLE = {'Wicket Keeper': 2, 'Batsman': 3, 'Bowler': 3}

budget = 50
squad = []
total_cost = 0

role_count = {'Wicket Keeper': 0, 'Batsman': 0, 'Bowler': 0}

for _, row in df_sorted.iterrows():

    if len(squad) == 15:
        break

    price = row['Price_in_Cr']
    role = row['Type']

    if total_cost + price > budget:
        continue

    remaining_slots = 15 - len(squad)

    unmet_roles = sum(
        max(0, MIN_ROLE[r] - role_count[r])
        for r in MIN_ROLE
    )

    if role in MIN_ROLE and role_count[role] < MIN_ROLE[role]:
        pass
    elif remaining_slots == unmet_roles:
        continue

    squad.append(row)
    total_cost += price

    if role in role_count:
        role_count[role] += 1


squad_df = pd.DataFrame(squad)

print("Total Cost:", total_cost)
print("Role Count:", role_count)
squad_df[['Player_Name','Type','Price_in_Cr','final_score']]


Total Cost: 19.949999999999996
Role Count: {'Wicket Keeper': 2, 'Batsman': 3, 'Bowler': 7}


,Player_Name,Type,Price_in_Cr,final_score
0,Mohsin Khan,Bowler,0.20,0.707402
1,Kuldeep Yadav,Bowler,2.00,0.745083
2,Rasikh Dar,Bowler,0.20,0.683333
3,B. Sai Sudharsan,All-Rounder,0.20,0.682153
4,David Miller,Batsman,3.00,0.725425
5,Maheesh Theekshana,Bowler,0.70,0.644958
6,Prerak Mankad,All-Rounder,0.20,0.614190
7,Devon Conway,Batsman,1.00,0.634918
8,Umesh Yadav,Bowler,2.00,0.652109
9,Mukesh Choudhary,Bowler,0.20,0.598327
